In [1]:
# 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

!pip install optuna -q

import pickle
import pandas as pd
import numpy as np
import optuna
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, f1_score
import warnings
warnings.filterwarnings('ignore')

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 7.6 MB/s eta 0:00:00


# 1. 데이터 로드

In [3]:
print("="*60)
print("📂 데이터 로드")
print("="*60)

LOAD_PATH = '/content/drive/MyDrive/preprocessed_data.pkl'

with open(LOAD_PATH, 'rb') as f:
    data_dict = pickle.load(f)

X_train = data_dict['X_train_clf']
X_val = data_dict['X_val_clf']
X_test = data_dict['X_test_clf']
y_train = data_dict['y_train_clf']
y_val = data_dict['y_val_clf']
y_test = data_dict['y_test_clf']

# Train + Val 합치기
X_train_full = pd.concat([X_train, X_val], axis=0).reset_index(drop=True)
y_train_full = pd.concat([y_train, y_val], axis=0).reset_index(drop=True)

print(f"✅ Train+Val: {X_train_full.shape}, Test: {X_test.shape}")

📂 데이터 로드
✅ Train+Val: (87026, 45), Test: (9670, 45)


In [4]:
# =============================================================================
# 스케일링
# =============================================================================
print("\n📌 스케일링 적용")

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_full)
X_test_scaled = scaler.transform(X_test)

print(f"✅ 스케일링 완료")


📌 스케일링 적용
✅ 스케일링 완료


# 2. Logistic Regression: 기본

In [5]:
print("\n" + "="*60)
print("📈 Logistic Regression (기본 버전)")
print("="*60)

lr_base = LogisticRegression(
    max_iter=1000,
    random_state=42,
    n_jobs=1
)

lr_base.fit(X_train_scaled, y_train_full)
lr_base_pred = lr_base.predict(X_test_scaled)

base_acc = accuracy_score(y_test, lr_base_pred)
base_f1 = f1_score(y_test, lr_base_pred, average='macro')

print(f"Accuracy: {base_acc:.4f}")
print(f"F1 Score (macro): {base_f1:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, lr_base_pred, target_names=['Bad', 'Standard', 'Good']))


📈 Logistic Regression (기본 버전)
Accuracy: 0.6552
F1 Score (macro): 0.6254

Classification Report:
              precision    recall  f1-score   support

         Bad       0.67      0.51      0.58      2749
    Standard       0.68      0.75      0.71      5150
        Good       0.56      0.61      0.59      1771

    accuracy                           0.66      9670
   macro avg       0.64      0.62      0.63      9670
weighted avg       0.66      0.66      0.65      9670



# 3. Logistic Regression: 최적화

In [6]:
print("\n" + "="*60)
print("📈 Logistic Regression (Optuna 최적화)")
print("="*60)

# 샘플링 (빠른 튜닝)
X_sample, _, y_sample, _ = train_test_split(
    X_train_scaled, y_train_full,
    train_size=0.3, random_state=42, stratify=y_train_full
)
print(f"✅ 튜닝용 샘플: {X_sample.shape}")

def objective_lr(trial):
    params = {
        'C': trial.suggest_float('C', 0.001, 100, log=True),
        'penalty': trial.suggest_categorical('penalty', ['l1', 'l2']),
        'solver': 'saga',  # l1, l2 둘 다 지원
        'class_weight': trial.suggest_categorical('class_weight', ['balanced', None]),
        'max_iter': 1000,
        'random_state': 42,
        'n_jobs': 1
    }

    model = LogisticRegression(**params)
    scores = cross_val_score(model, X_sample, y_sample, cv=3, scoring='f1_macro', n_jobs=1)
    return scores.mean()

optuna.logging.set_verbosity(optuna.logging.WARNING)
study_lr = optuna.create_study(direction='maximize')
study_lr.optimize(objective_lr, n_trials=20, show_progress_bar=True)

print(f"\n✅ 최적 파라미터: {study_lr.best_params}")
print(f"✅ 최고 CV F1: {study_lr.best_value:.4f}")

# 최적 파라미터로 학습
best_params = study_lr.best_params
best_params['solver'] = 'saga'
best_params['max_iter'] = 1000
best_params['random_state'] = 42
best_params['n_jobs'] = 1

lr_opt = LogisticRegression(**best_params)
lr_opt.fit(X_train_scaled, y_train_full)
lr_opt_pred = lr_opt.predict(X_test_scaled)

opt_acc = accuracy_score(y_test, lr_opt_pred)
opt_f1 = f1_score(y_test, lr_opt_pred, average='macro')

print(f"\nAccuracy: {opt_acc:.4f}")
print(f"F1 Score (macro): {opt_f1:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, lr_opt_pred, target_names=['Bad', 'Standard', 'Good']))


📈 Logistic Regression (Optuna 최적화)
✅ 튜닝용 샘플: (26107, 45)


  0%|          | 0/20 [00:00<?, ?it/s]


✅ 최적 파라미터: {'C': 0.0030527033936655467, 'penalty': 'l1', 'class_weight': 'balanced'}
✅ 최고 CV F1: 0.6512

Accuracy: 0.6543
F1 Score (macro): 0.6489

Classification Report:
              precision    recall  f1-score   support

         Bad       0.61      0.68      0.64      2749
    Standard       0.83      0.57      0.68      5150
        Good       0.50      0.85      0.63      1771

    accuracy                           0.65      9670
   macro avg       0.64      0.70      0.65      9670
weighted avg       0.70      0.65      0.66      9670



# 4. 결과 비교 및 모델 저장

In [7]:

# =============================================================================
# 비교 결과
# =============================================================================
print("\n" + "="*60)
print("📊 기본 vs 최적화 비교")
print("="*60)

comparison = pd.DataFrame({
    'Model': ['Logistic Regression (기본)', 'Logistic Regression (Optuna)'],
    'Accuracy': [base_acc, opt_acc],
    'F1 (macro)': [base_f1, opt_f1]
})
print(comparison.to_string(index=False))


📊 기본 vs 최적화 비교
                       Model  Accuracy  F1 (macro)
    Logistic Regression (기본)  0.655222    0.625392
Logistic Regression (Optuna)  0.654292    0.648870


In [8]:
# =============================================================================
# 모델 저장
# =============================================================================
SAVE_PATH = '/content/drive/MyDrive/best_lr_model.pkl'

best_model = lr_opt if opt_f1 > base_f1 else lr_base
best_params_save = best_params if opt_f1 > base_f1 else {'default': True}

save_dict = {
    'model': best_model,
    'best_params': best_params_save,
    'scaler': scaler
}

with open(SAVE_PATH, 'wb') as f:
    pickle.dump(save_dict, f)

print(f"\n✅ 모델 저장 완료: {SAVE_PATH}")


✅ 모델 저장 완료: /content/drive/MyDrive/best_lr_model.pkl
